# Model calibration
This notebook contains a calibration of the model for the current number of agents. Usually ABMs are not linearly scalable and therefore this procedure should be followed for model calibration, to avoid unnecessary long start-up time of the model and thus a wastage of computing power. Because the RIVM stakeholders mentioned during one of the final meetings that they were especially interested in the process of model-building, this notebook is an annex to assist for when the model is built. Note that in case this model will be extended, this notebook will require to be extended, since it only includes current variables. However, most certainly the process stays the same.

Calibration outcomes of the model need to be implemented directly in the agents manually. Due to ABMs being stochastic, initial values will not be perfect but provide a good starting point. First we will be importing the model in the notebook and set it up in such a way that all policies and scenarios are turned off to see whether we can get the model in some sort of equilibrium.

In [1]:
from model.cepai_model import *

levers = {
    "L1": 0.0,  # Minimal requirement for reused parts
    "L2": 0.0,  # Minimal requirement for high-quality plastic
    "L3": 0.0,  # Use better solvable cohesives
    "L4": 1.0,  # Include externality for virgin plastic
    "L5": 0.05   # Minimal requirement for recyclate, calibrated for 5% according to Volvo.
        }

uncertainties = {
    "X1": 0.0,  # Annual increase factor of oil price
    "X2": 0.0,  # Annual probability for global oil shock
    "X3": 0.0   # Annual increase of recycling efficiency
        }

In [2]:
model = CEPAIModel(levers=levers, uncertainties=uncertainties)

Simulation starting...


In [3]:
model_run = model.run(steps=50, time_tracking=True)


KeyboardInterrupt



In [16]:
model_run

NameError: name 'model_run' is not defined

In [3]:
for _ in range(50):
    #print(_)
    model.step()

In [4]:
for RC in model.all_agents[Recycler]:
    print(RC.stock[Component.RECYCLATE_HIGH])
    print(RC.stock[Component.RECYCLATE_LOW])
    # # part = RC.stock[Component.PARTS_FOR_RECYCLER][0]
    # # print(part.plastic_ratio)
    # print(RC.stock[Component.PARTS_FOR_RECYCLER])
    print(RC.stock[Component.CARS_FOR_RECYCLER])
    for part in RC.stock[Component.PARTS_FOR_RECYCLER]:
        print(part.state)
        print(part.plastic_ratio)

6485.58134265207
23.20039292136384
[<model.bigger_components.Car object at 0x00000232D69CB190>, <model.bigger_components.Car object at 0x00000232D67AED70>, <model.bigger_components.Car object at 0x00000232D614D540>, <model.bigger_components.Car object at 0x00000232D632C370>, <model.bigger_components.Car object at 0x00000232D67AD1E0>, <model.bigger_components.Car object at 0x00000232D6859F30>, <model.bigger_components.Car object at 0x00000232D61773A0>, <model.bigger_components.Car object at 0x00000232D649FE20>, <model.bigger_components.Car object at 0x00000232D69CA410>, <model.bigger_components.Car object at 0x00000232D5DC06A0>, <model.bigger_components.Car object at 0x00000232D6235DB0>, <model.bigger_components.Car object at 0x00000232D6559930>, <model.bigger_components.Car object at 0x00000232D6B9B430>, <model.bigger_components.Car object at 0x00000232D649FD00>, <model.bigger_components.Car object at 0x00000232D6B35C90>, <model.bigger_components.Car object at 0x00000232D671E800>, <mod

In [5]:
for PM in model.all_agents[PartsManufacturer]:
    print(PM.demand[Component.PARTS])
    print(PM.stock[Component.VIRGIN])
    print(PM.stock[Component.RECYCLATE_LOW])
    print(PM.stock[Component.RECYCLATE_HIGH])
    print(PM.plastic_ratio)

110
861.9486849922214
9.233691085622105
18.81762392202549
{<Component.VIRGIN: 1>: 0.9459789669064507, <Component.RECYCLATE_LOW: 2>: 0.05402103309354936, <Component.RECYCLATE_HIGH: 3>: 0.0}
705
913.6149496871916
10.671654193688184
22.71339611928255
{<Component.VIRGIN: 1>: 0.9451180699216319, <Component.RECYCLATE_LOW: 2>: 0.05488193007836804, <Component.RECYCLATE_HIGH: 3>: 0.0}
765
1072.4793628311525
41.160769162549414
2.359868006567491
{<Component.VIRGIN: 1>: 0.9464743879278855, <Component.RECYCLATE_LOW: 2>: 0.053525612072114516, <Component.RECYCLATE_HIGH: 3>: 0.0}


As we can see above, simply no parts are produced because there is not enough low quality recyclate!

In [6]:
for CM in model.all_agents[CarManufacturer]:
    # print(CM.stock)
    print(CM.demand)
    print(CM.sold_volume)

{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 102, <Component.CARS: 6>: 4}
{'last': 0, 'second_last': 0}
{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 101, <Component.CARS: 6>: 22}
{'last': 0, 'second_last': 0}
{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 101, <Component.CARS: 6>: 17}
{'last': 0, 'second_last': 0}
{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 172, <Component.CARS: 6>: 43}
{'last': 42, 'second_last': 57}


In [9]:
model.step()

In [10]:
counter = 0
counter_to_buy = 0
counter_to_retrieve = 0
for agent in model.all_agents[User]:
    if agent.stock[Component.CARS]:
        counter += 1
        car = agent.stock[Component.CARS][0]
        print(car.state)
    else:
        print(agent.demand[Component.CARS])
        if agent.demand[Component.CARS] == 1:
            counter_to_buy += 1

        else:
            counter_to_retrieve += 1

CarState.FUNCTIONING
CarState.END_OF_LIFE
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
1
CarState.FUNCTIONING
1
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.BROKEN
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.END_OF_LIFE
CarState.FUNCTIONING
1
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarSt

In [11]:
print(counter)
print(counter_to_buy)
print(counter_to_retrieve)
print(counter+counter_to_retrieve+counter_to_buy)

949
51
0
1000


In [17]:
cars_tb_repaired = 0
for garage in model.all_agents[Garage]:
    cars_tb_repaired += len(garage.stock[Component.CARS])
    print("Parts in stock")
    print(len(garage.stock[Component.PARTS]))
    print("Cars tb repaired")
    print(len(garage.stock[Component.CARS]))
    print("Outstanding customer base of garage:")
    print(garage.customer_base)
    # print("Cars tb recycled")
    # print(len(garage.stock[Component.CARS_FOR_RECYCLER]))
    # print("Cars tb dismantled")
    # print(len(garage.stock[Component.CARS_FOR_DISMANTLER]))

Parts in stock
228
Cars tb repaired
0
Outstanding customer base of garage:
{<model.bigger_components.Car object at 0x0000022421C1BF40>: <model.agents.User object at 0x0000022421C1BF10>, <model.bigger_components.Car object at 0x0000022421C75900>: <model.agents.User object at 0x0000022421C758D0>, <model.bigger_components.Car object at 0x0000022421D0E9E0>: <model.agents.User object at 0x0000022421D0E9B0>, <model.bigger_components.Car object at 0x0000022421D99FC0>: <model.agents.User object at 0x0000022421D99F90>, <model.bigger_components.Car object at 0x0000022421E55ED0>: <model.agents.User object at 0x0000022421E55EA0>, <model.bigger_components.Car object at 0x0000022421EBAE90>: <model.agents.User object at 0x0000022421EBAE60>, <model.bigger_components.Car object at 0x0000022421F175B0>: <model.agents.User object at 0x0000022421F17580>, <model.bigger_components.Car object at 0x0000022421FA7100>: <model.agents.User object at 0x0000022421FA70D0>, <model.bigger_components.Car object at 0x000

In [37]:
for PM in model.all_agents[PartsManufacturer]:
    # PM.stock[Component.PRODUCED_PARTS] = 0
    # print(PM.stock[Component.PRODUCED_PARTS])
    print(PM.plastic_ratio)

0
{<Component.VIRGIN: 1>: 0.9418815845799824, <Component.RECYCLATE_LOW: 2>: 0.0581184154200176, <Component.RECYCLATE_HIGH: 3>: 0.0}
0
{<Component.VIRGIN: 1>: 0.9388474718074881, <Component.RECYCLATE_LOW: 2>: 0.061152528192511914, <Component.RECYCLATE_HIGH: 3>: 0.0}
0
{<Component.VIRGIN: 1>: 0.9445457975319946, <Component.RECYCLATE_LOW: 2>: 0.055454202468005385, <Component.RECYCLATE_HIGH: 3>: 0.0}


In [38]:
for PM in model.all_agents[PartsManufacturer]:
    # print(PM.stock[Component.PARTS])
    # print(PM.stock[Component.PRODUCED_PARTS])
    print(PM.stock[Component.VIRGIN])
    print(PM.stock[Component.RECYCLATE_HIGH])
    print(PM.stock[Component.RECYCLATE_LOW])

36288.23559538797
0.0
0.02574460968089462
34938.88539794366
0.0
0.004722662861180241
28842.71816335236
0.0
0.047588307604668194
